### Validation for Empty Video Folder

In [78]:
import glob
folder = 'Z:/VIDEOS'
# video_list = glob.glob(folder + '/*/Video/**/*.asf')
# video_list

ids = os.listdir(folder)

for idx in ids:
    files = glob.glob(folder + '/' + idx + '/Video/**/*.asf')
    if len(files) < 1:
        print(f'idx: {idx}, files: {len(files)}')

idx: 1003 1004-nonAI, files: 0
idx: 1005-nonAI, files: 0


In [1]:
import os
import json
import random
import metaData
import argparse
from tqdm import tqdm
import datetime
import pandas as pd


In [ ]:
json_file = 'Datafiles/data_storage_Jan2023.json'
with open(json_file) as file:
    data = json.load(file)

In [ ]:
days_list = {}
days = []
for i, d in enumerate(data['data'][:500]):
    if d['id'] in days_list.keys():
        days_list[d['id']].append(d['day'])
    elif d['id'] not in days_list.keys():
        days_list[d['id']] = list()
        days_list[d['id']].append(d['day'])
    # print(f"Index: {i}, Data: {d['day']}")
    # days.append(d['day'])

days_list_sorted = {k: sorted(v) for k, v in days_list.items()}

In [ ]:
df = pd.read_json(json_file)
dt = pd.DataFrame(df['data'], columns=['id', 'day', 'duration'])


jn = pd.json_normalize(df['data'])


In [ ]:
col_list = []
ts_list = []
for col, row in jn.iloc[:10].iterrows():
    for idx in row:
        # col_list.append(idx)
        if type(idx) == list and len(idx) > 0:
            for item in idx:
                ts_list.append(item)
        else:
            col_list.append(idx)
            if ts_list != []:
                col_list.append(ts_list)
                ts_list = []
            

In [ ]:
jn.columns

ts_cols = ['yawn.timestamp',
           'smoking.timestamp',
           'mobilephone.timestamp',
           'distraction.timestamp',
           'eyeclosing.timestamp',
           'crossinglane.timestamp',
           'nearcollision.timestamp',
           'stopsign.timestamp',
           'redlight.timestamp',
           'pedestrian.timestamp',
           ]


Index(['id', 'day', 'duration', 'yawn.total', 'yawn.timestamp',
       'smoking.total', 'smoking.timestamp', 'mobilephone.total',
       'mobilephone.timestamp', 'distraction.total', 'distraction.timestamp',
       'eyeclosing.total', 'eyeclosing.timestamp', 'crossinglane.total',
       'crossinglane.timestamp', 'nearcollision.total',
       'nearcollision.timestamp', 'stopsign.total', 'stopsign.timestamp',
       'redlight.total', 'redlight.timestamp', 'pedestrian.total',
       'pedestrian.timestamp'],
      dtype='object')

### Data Aggregate

In [ ]:
#  sort a list of dictionaries by value1 and value2
def sort_list_of_dict_by_value(list_of_dict, value1, value2):
    return sorted(list_of_dict, key=lambda k: (k[value1], k[value2]))


# get next 90 days from a given date
# return a single date for next 90 days
def get_next_90_days(start_date):
    yyyy = start_date[:4]
    mm = start_date[4:6]
    dd = start_date[6:]

    converted_date = datetime.datetime(int(yyyy), int(mm), int(dd))
    next_months = converted_date + datetime.timedelta(days=90)
    return str(next_months)[:10].replace('-', '')

def get_next_90_days_data_from_sorted_list_of_dict(list_of_dict, value1, value2):
    next_90_days = []
    for i, d in enumerate(list_of_dict):
        if i == len(list_of_dict)-1:
            break
        next_90_days.append(get_next_90_days(d[value2]))
    return next_90_days

# calculate number of day between two dates
def days_between(d1, d2):
    d1 = datetime.datetime.strptime(d1, "%Y%m%d")
    d2 = datetime.datetime.strptime(d2, "%Y%m%d")
    return abs((d2 - d1).days)

# days_between('20200301', '20200102')

In [ ]:
sl = sort_list_of_dict_by_value(data['data'], 'id', 'day')
d = get_next_90_days_data_from_sorted_list_of_dict(sl, 'id', 'day')

### Extracting Data Based on Timestamp

In [2]:
# filePath = 'Z:/VIDEOS/1010/Disk_files/debug/'
def extract_indices_from_log(filePath,file):
    alarmsDict = {'NOBODY': 0, 'LOOKING_DOWN': 0, 'SMOKING': 0, 'CALLING':0, 'LDW': 0, 'EYE_CLOSED': 0, 'LDW_R': 0, 'LDW_L':0, 'FCW':0, 'camera cover!':0, 'infrared block!': 0 }
    alarmsTimeStamp = {'NOBODY': [], 'LOOKING_DOWN': [], 'SMOKING': [], 'CALLING':[], 'LDW': [], 'EYE_CLOSED': [], 'LDW_R': [], 'LDW_L':[], 'FCW':[], 'camera cover!':[], 'infrared block!': [] }
    # alarmsDict = {'alarm_type 5': 0, 'alarm_type 4': 0, 'alarm_type 3': 0, 'alarm_type 1': 0, 'alarm_type 2': 0, 'alarm_type 17': 0, 'alarm_type 18': 0, 'alarm_type 27':0, 'alarm_type 16':0, 'alarm_type 9':0, 'alarm_type 7': 0 }
    # alarmsTimeStamp = {'alarm_type 5': [], 'alarm_type 4': [], 'alarm_type 3': [], 'alarm_type 1': [], 'alarm_type 2': [], 'alarm_type 17': [], 'alarm_type 18': [], 'alarm_type 27':[], 'alarm_type 16':[], 'alarm_type 9':[], 'alarm_type 7': [] }
    
    error_files = []
    # print(f"Processing debug file {file}")
    with open(os.path.join(filePath, file), 'r') as logFile:
        lines = logFile.read().splitlines()
        for line in lines:
            words = line.split(" ")
            """
            Retrieve alarm will be here
            """
            for alarm in alarmsDict.keys():
                if alarm in words:
                    alarmsDict[alarm] += 1
                    # below is date extraction so needs to be corrected****** --> currently is this -->[20210504-140826][03][baseIVS]
                    try:
                        timestamp = "".join(words[0].split('[')[1].split('-')).strip(']')
                        alarmsTimeStamp[alarm].append(timestamp)
                    except Exception:
                        error_files.append(file)

        # logFile.close()
        # print(alarmsDict)
    return alarmsDict, alarmsTimeStamp, error_files

In [3]:
excluded_list = ['1003 1004','1031 1017','1011 1012','1043 1044','1013 1014','1003 1004-nonAI', '1005-nonAI', '1073', '2062']
ids = []
for item in os.listdir('Z:/VIDEOS/'):
    if item not in excluded_list:
        ids.append(item)

In [9]:
jsonfile = []
mapping = {'NOBODY': 0,
           'LOOKING_DOWN': 1,
           'SMOKING': 2, 
           'CALLING': 3, 
           'LDW': 5, 
           'EYE_CLOSED': 4, 
           'LDW_R': 5, 
           'LDW_L':5, 
           'FCW':6, 
           'camera cover!':0, 
           'infrared block!': 0 }
    
for each_driver in tqdm(ids):
    logFolder = f'Z:/VIDEOS/{each_driver}/Disk_files/debug/'
    for logfile in os.listdir(logFolder):
        driverid = logFolder.split('/')[2]
        
        # if logfile.endswith('.log'):
        alarmdict, alarmtimestamp, err = extract_indices_from_log(logFolder, logfile)
        for key, value in alarmtimestamp.items():
            for ts in value:
                if key != 'NOBODY':
                    id = driverid
                    alarm_type = key
                    date = ts[:9]
                    
                    transform_dict = {'id': id, 'date': date, 'timestamp': ts, 'type': mapping[alarm_type]}
                    jsonfile.append(transform_dict)

with open('./Datafiles/Timestamps_data.json', 'w') as jsonf:
    json.dump(jsonfile, jsonf)    
        
        

100%|██████████| 40/40 [06:11<00:00,  9.30s/it]
